<h2> EHRA Dashboard </h2>

First we have to generate simulated data to fill the scattermap. We'll store it in the dataframe <code>airQual</code>

In [1]:
## Use code from Annie

# Sanity check for valid data structure 
# airQual.head()

Before getting the plot to show, need to sign up for mapbox account to get mapbox token: https://account.mapbox.com/

In [8]:
import plotly.express as px
px.set_mapbox_access_token("pk.eyJ1IjoiemlxaW50aWFuIiwiYSI6ImNsMWdzb2JkNjFlZm4zcG54NWhzaWVtdXkifQ.Bewl0ISItyYwAGSAFPOQRA")
df = px.data.carshare()
fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon", color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,  width=700, height=800)
fig.show()

PermissionError: [Errno 1] Operation not permitted

If we wanted to just export this interactive plotly figure at this point as an html file, we can simply use the following snippet of code:

In [3]:
## fig.write_html("testing.html")

<h3> Using Dash </h3>

Here we're relying on JupyterDash to view the dashboard within our Jupyter notebook. Here I'm only implementing the layout without any of the callback functionality so that we have framework to build the visualization.

We'll still need to figure out a way to make the map full size and figure out what kind of filtering functionality we might want. Later we'll need focus on customizing the actual visuals.

In [80]:
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import os
import base64

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

logo = os.getcwd() + '/assets/EHRA_logo.jpeg'
encoded_image = base64.b64encode(open(logo, 'rb').read())

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "1rem 1rem",
    "background-color": "#f8f9fa",
}

FIGURE_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "margin-left": "16rem",
    "margin-right": "1rem"
}

NEWS_STYLE = {
    "position": "fixed",
    "top": "0rem",
    "padding-top": "3rem",
    "padding-left": "1rem",
    "left": 0,
    "bottom": 0,
    "margin-left": "61rem",
    "margin-right": "1rem",
    "width": "40rem",
    "background-color": "#f8f9fa",
}

sidebar = html.Div(
    [
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '220px'}),
        html.H4('Pollutant'),
        dcc.RadioItems(id = 'radio-item-1',
                        options = [dict(label = 'NO2', value = 'A'),
                                    dict(label = 'VOC', value = 'B'),
                                    dict(label = 'pm1', value = 'C'),
                                    dict(label = 'pm10', value = 'D'),
                                    dict(label = 'pm25', value = 'E')],
                        value = 'A',
                        labelStyle={'display': 'block'}),

        html.P(['Abbreviations', html.Br(),\
            'PM: Particulate Matter', html.Br(),\
            'NO2: Nitrogen Dioxide', html.Br(),\
                'VOC: Volatine Organic Compound', html.Br(),\
                    'PM10: PM<10um diameter', html.Br(),\
                        'PM25: PM<2.5um diameter']),
        
        html.H4(['Select hotspot:']),

        dcc.Dropdown(
            ['Elementary School', 'Factory', 'Middle School', 'Park'],
            'Select hotspots',
            id='hotspot-dropdown'
        ),
        html.P('These hotspots were selected based on the air quality pollutant data released to the government.'),
        dbc.Button("Upload my Flow Monitor Data", color="primary", className="me-1", 
        href='https://en.wikipedia.org/wiki/Main_Page', target='_blank'),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(dcc.Graph(id = 'main-graph', figure = fig), style = FIGURE_STYLE)
news = html.Div(html.H4('Related News & Information'), style = NEWS_STYLE)
app.layout = html.Div([sidebar, content, news])

@app.callback(
    Output('main-graph', 'figure'),
    Input('radio-item-1', 'value'))
def update_figure(color_scheme):
    palette = {'A': px.colors.cyclical.HSV,
             'B': px.colors.cyclical.Phase,
             'C': px.colors.cyclical.Twilight,
             'D':  px.colors.cyclical.mrybm,
             'E': px.colors.cyclical.mygbm}
    fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon",     color="peak_hour", size="car_hours",
                  color_continuous_scale=palette[color_scheme], size_max=15, zoom=10,  width=700, height=800)

    fig.update_layout(coloraxis_colorbar=dict(
    title="NO2",
    ))
    return fig


## Deploy app -------------------------------------------------------------
del app.config._read_only["requests_pathname_prefix"]
app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/


/Users/branglab/Desktop/Misc/EunSeon/EHRA/env/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Deleting app configuration is a weird fix for JupyterDash update that makes some parts of it no longer completely compatible with Dash. We Run with  <code>  mode = "inline" </code> so we can view the dashboard within the Jupyter notebook itself.

In [82]:
#del app.config._read_only["requests_pathname_prefix"]
#app.run_server(mode="inline")

# Run following code in the terminal to generate requirements.txt
#jupyter nbconvert --output-dir="./reqs" --to script EHRA_Dashboard.ipynb
#cd reqs
#pipreqs